In [9]:
import dask.dataframe as dd
import pandas as pd
from IPython.display import HTML
from pigeonXT import annotate
from datetime import datetime
from ast import literal_eval

#Return the HTML code with matched table, highlight the row where the match was located, and add an additional table with match information
def create_display_table(row):
    rowIndex = int(row[1]['rowIndex'])
    css = f'<head><style>.wikitr:nth-child({rowIndex+1}) {{background-color: #bdcebe !important;}}</style></head>'
    url = f'<a href="https://simple.wikipedia.org/wiki/{row[1]["pageTitle"]}">Visit wikipedia page</a>'
    header = f"<table><tr><td>Page title</td><td> {row[1]['pageTitle']}</td><td> tableIndex: {row[1]['tableIndex']} rowIndex: {row[1]['rowIndex']} </td><td> </td><td>{url}</td></tr>"
    
    c1 = int(row[1]['c1'])
    c2 = int(row[1]['c2'])
    wikibase_item = int(row[1]['wikibase_item'])
    year = int(row[1]['year'])
    wikidata_ids_str = row[1]['wikidata_ids']
    wiki_names = row[1]['wikidata_names']
    ents_title = row[1]['ents_title']
    
    p = row[1]['p']
    pq = row[1]['pq']
    
    index1 = wikidata_ids_str.index(c1)
    index2 = wikidata_ids_str.index(c2)
    try:
        index3 = wikidata_ids_str.index(wikibase_item)
        url = f'<a href="https://www.wikidata.org/wiki/Q{wikibase_item}">Visit wikidata page</a>'
        info5 = f'<tr><td>Qualifier value</td><td> Q{wikibase_item}</td><td> {wiki_names[index3]} </td><td> {ents_title[index3]} </td><td>{url}</td></tr></table>'
    except Exception:
        info5 = '</table>'
        if int(year) > 0:
            index3 = wikidata_ids_str.index(-9)
            info5 = f'<tr><td>Qualifier value</td><td> year</td><td> {wiki_names[index3]} </td><td>Year found in cell text</td><td> </td</tr></table>'


    url = f'<a href="https://www.wikidata.org/wiki/Q{c1}">Visit wikidata page</a>'
    info1 = f'<tr><td>Subject</td><td> Q{c1}</td><td> {wiki_names[index1]} </td><td> {ents_title[index1]} </td><td>{url}</td></tr>'
    url = f'<a href="https://www.wikidata.org/wiki/Property:P{p}">Visit wikidata page</a>'
    info2 = f'<tr><td>Property</td><td> P{p}</td><td> {property_dict["P"+str(p)]} </td><td> </td><td>{url}</td></tr>'
    
    url = f'<a href="https://www.wikidata.org/wiki/Q{c2}">Visit wikidata page</a>'
    info3 = f'<tr><td>Object</td><td> Q{c2}</td><td> {wiki_names[index2]} </td><td> {ents_title[index2]} </td><td>{url}</td></tr>'
    url = f'<a href="https://www.wikidata.org/wiki/Property:P{pq}">Visit wikidata page</a>'
    info4 = f'<tr><td>Qualifier property</td><td> P{pq}</td><td> {property_dict["P"+str(pq)]} </td><td> </td><td>{url}</td></tr>'
    
    wiki_table = str(row[1]['pageHTML']).replace('<tr>', '<tr class="wikitr">')
    return HTML(header + info1 + info2 + info3 + info4 + info5 + '<!DOCTYPE html><html>'+ css + wiki_table + "</html>")

def save_annotations_to_csv():
    if all(annotations.changed):
        file_name = "./annotations/annotations/" + "annotations_" + str(datetime.now()).replace(":", "").replace(" ","_") + ".csv"
        annotation_df = df_pandas.join(annotations)
        annotation_df.to_csv(file_name)
        print("Annotations saved to csv", file_name)
        return file_name
    else:
        print("Annotations are not completed yet")

In [10]:
#Specify files names and directories
file_name = "simple_100_all"
file_name = "./annotations/files/" + file_name + ".csv"
df_pandas = pd.read_csv(file_name, encoding="utf-8")

df_pandas['wikidata_ids'] = df_pandas['wikidata_ids'].apply(literal_eval)
df_pandas['wikidata_names'] = df_pandas['wikidata_names'].apply(literal_eval)
df_pandas['ents_title'] = df_pandas['ents_title'].apply(literal_eval)

#Load property labels to list
property_dict = {}
f = open('wikidata-prop-label.tsv','r',encoding = 'utf-8')
for line in f:
    property_id, property_label = [elem.strip() for elem in line.split('\t')]
    property_dict[property_id] = property_label
f.close()

In [11]:
#Start annotation tool
annotations = annotate(
  [table for table in df_pandas.iterrows()],
  options=['N-ary', 'Not N-ary', 'Incorrect formatting', 'Skip'],
  display_fn=lambda row:display(create_display_table(row))
)

HTML(value='0 of 100 Examples annotated, Current Position: 0 ')

Output()

In [ ]:
#Saves annotations to CSV
file_name = save_annotations_to_csv()

Annotations are not completed yet
